In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras import layers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2

In [2]:
# local paths 
training_dir = "train_images_inferred_augmented/train_images" #This is the inferred augmented dir
testing_dir = "test_images" 

In [3]:
# Import data
class_names_dict = np.load("class_names.npy", allow_pickle=True).item()
attributes_npy = np.load("attributes.npy", allow_pickle=True)
attributes_df = pd.read_csv("attributes.txt", header=None, index_col = 0, sep=" |::", names=("attribute", "value"), engine='python')
training_labels_df = pd.read_csv("train_images.csv", header = 0)
test_images_paths = pd.read_csv("test_images_path.csv", header = 0)
test_images_sample = pd.read_csv("test_images_sample.csv", header = 0)

In [4]:
model_VGG = tf.keras.models.load_model('best_model_33.h5')
model_DenseNet = tf.keras.models.load_model('best_model_densenet_43.h5')
model_InceptionV3 = tf.keras.models.load_model('best_model_49.h5')


2023-12-14 15:12:28.214125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [5]:


resize_dim = (224, 224)

def read_image(image_path):
    image = cv2.imread(testing_dir+image_path)
    img_resized = cv2.resize(image, resize_dim)
    img_normalized = img_resized / 255.0
    return img_normalized


test_images_paths['pixel_array'] = test_images_paths['image_path'].apply(read_image)

test_images = np.array(test_images_paths['pixel_array'].tolist())


## VGG Predicitons

In [6]:
predictions = model_VGG.predict(test_images)

2023-12-14 13:38:09.189546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


 10/125 [=>............................] - ETA: 2s

2023-12-14 13:38:11.828585: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


125/125 [==============================] - 8s 19ms/step


In [7]:
predicted_labels_vgg = np.argmax(predictions, axis=1)

In [8]:
predicted_labels_vgg

array([ 69, 119,  74, ..., 102,  39, 118])

## Inception Predictions

In [9]:
predictions = model_InceptionV3.predict(test_images)

125/125 [==============================] - 4s 13ms/step


In [10]:
predicted_labels_Inception = np.argmax(predictions, axis=1)

In [11]:
predicted_labels_Inception

array([ 67,  37,  74, ..., 135,  62, 111])

## DenseNet Predictions

In [12]:
predictions = model_DenseNet.predict(test_images)

125/125 [==============================] - 5s 19ms/step


In [13]:
predicted_labels_DenseNet = np.argmax(predictions, axis=1)

In [14]:
predicted_labels_DenseNet

array([ 69,  54,  74, ..., 135,  53, 111])

## ENSEMBLE

In [15]:
def ensemble(array1, array2, array3):
    equal_positions = []

    for i in range(len(array1)):
        if array1[i] == array2[i]:
            equal_positions.append(array1[i])
        else:
            equal_positions.append(array3[i])

    return equal_positions



ensembled_labels = ensemble(predicted_labels_DenseNet, predicted_labels_vgg,predicted_labels_Inception )


In [16]:
ensembled_labels = np.array(ensembled_labels)

In [17]:
ensembled_labels.shape

(4000,)

In [18]:
test_images_sample['label'] = ensembled_labels

In [19]:
csv_file_path = 'kaggle_file.csv'

test_images_sample.to_csv(csv_file_path, index=False)

print(f'DataFrame has been saved to {csv_file_path}')

DataFrame has been saved to kaggle_file.csv


## Heat Map for predictions

In [1]:
# This loads the training images from the balanced augmented training dir

images = []
labels = []

resize_dim = (224, 224)

# Iterate through subdirectories
for subdir in range(1, 201):
    subdir_path = os.path.join(training_dir, str(subdir))
    counter = 0;
    # Iterate through files in each subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith(".jpg"):
            
            # Read images
            image_path = os.path.join(subdir_path, filename)
            img = cv2.imread(image_path)
            
            # Fix images
            img_resized = cv2.resize(img, resize_dim)
            img_normalized = img_resized / 255.0
            
            #Load images
            images.append(img_normalized)
            labels.append(subdir)
            
            counter += 1
            if counter >= 3:
                break;

# Convert lists to numpy arrays
images_array = np.array(images)
labels_array = np.array(labels)

IndentationError: expected an indented block after 'if' statement on line 29 (210063135.py, line 32)

In [ ]:
df = pd.DataFrame({'true_labels': array1, 'Column2': array2})